In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
team_stats = pd.read_csv("datasets/team_stats.csv")
standings = pd.read_csv("datasets/standings.csv")
fixtures = pd.read_csv("datasets/fixtures.csv")

In [13]:
# Ellenőrizzük, hogy biztosan léteznek-e a megfelelő oszlopok
print("Team stats oszlopok:")
print(team_stats.columns.tolist())
print("\nStandings oszlopok:")
print(standings.columns.tolist())

Team stats oszlopok:
['team', 'players', 'age', 'possession', 'goals', 'assists', 'penalty_kicks', 'penalty_kick_attempts', 'yellows', 'reds', 'expected_goals', 'expected_assists', 'progressive_carries', 'progressive_passes']

Standings oszlopok:
['rank', 'team', 'win', 'loss', 'draw', 'goals', 'conceded', 'points', 'last5', 'top_scorer', 'keeper']


In [23]:
# A modellhez használt 6 feature kiválasztása

features = [
    "goals",               # lőtt gólok
    "assists",             # gólpasszok
    "possession",          # labdabirtoklás
    "expected_goals",      # xG
    "expected_assists",    # xA
    "progressive_passes"   # előrehaladó passzok
]

X = team_stats[features]

# Az első 5 sor megjelenítése
X.head()


,goals,assists,possession,expected_goals,expected_assists,progressive_passes
0,65,53,56.8,57.6,43.2,1764
1,56,45,51.0,55.7,41.5,1326
2,55,39,48.1,62.4,42.6,1438
3,64,43,47.8,57.6,41.3,1318
4,60,40,52.0,56.5,39.7,1478


In [27]:
#Ellenőrzés
print("Használt feature-ök száma:", len(features))
print("Feature nevek:", features)



Használt feature-ök száma: 6
Feature nevek: ['goals', 'assists', 'possession', 'expected_goals', 'expected_assists', 'progressive_passes']
